# Install dependencies into condacolab environment

In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [3]:
!conda --version

conda 23.11.0


In [4]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [5]:
!conda install -y -c bioconda bamtools bedtools samtools

Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: | / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.4.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [6]:
!pip install modisco-lite

# Load and process data files

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!wget http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
!gunzip hg38.fa.gz
!samtools faidx hg38.fa

--2024-05-07 21:14:06--  http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘hg38.fa.gz’

hg38.fa.gz          100%[===================>] 938.09M   116MB/s    in 8.3s    

2024-05-07 21:14:15 (113 MB/s) - ‘hg38.fa.gz’ saved [983659424/983659424]

gzip: hg38.fa already exists; do you wish to overwrite (y or n)? ^C
^C


In [110]:
#alternative hg38 download
#ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.15_GRCh38/seqs_for_alignment_pipelines.ucsc_ids/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz

In [9]:
# from https://github.com/calico/basenji/blob/master/basenji/dna_io.py
def dna_1hot(seq, seq_len=None, n_uniform=False, n_sample=False):
  """ dna_1hot
    Args:
      seq:       nucleotide sequence.
      seq_len:   length to extend/trim sequences to.
      n_uniform: represent N's as 0.25, forcing float16,
      n_sample:  sample ACGT for N

    Returns:
      seq_code: length by nucleotides array representation.
    """
  if seq_len is None:
    seq_len = len(seq)
    seq_start = 0
  else:
    if seq_len <= len(seq):
      # trim the sequence
      seq_trim = (len(seq) - seq_len) // 2
      seq = seq[seq_trim:seq_trim + seq_len]
      seq_start = 0
    else:
      seq_start = (seq_len - len(seq)) // 2

  seq = seq.upper()

  # map nt's to a matrix len(seq)x4 of 0's and 1's.
  if n_uniform:
    seq_code = np.zeros((seq_len, 4), dtype='float16')
  else:
    seq_code = np.zeros((seq_len, 4), dtype='bool')

  for i in range(seq_len):
    if i >= seq_start and i - seq_start < len(seq):
      nt = seq[i - seq_start]
      if nt == 'A':
        seq_code[i, 0] = 1
      elif nt == 'C':
        seq_code[i, 1] = 1
      elif nt == 'G':
        seq_code[i, 2] = 1
      elif nt == 'T':
        seq_code[i, 3] = 1
      else:
        if n_uniform:
          seq_code[i, :] = 0.25
        elif n_sample:
          ni = random.randint(0,3)
          seq_code[i, ni] = 1

  return seq_code

In [10]:
import numpy as np

rep = np.load('/content/drive/MyDrive/cs294_rep_attributions/rep1.npz', mmap_mode='r')
attr = rep['grads']
starts = rep['starts'].flatten()
ends = rep['ends'].flatten()
chroms = rep['chroms'].flatten()

In [11]:
!pip install pysam

In [12]:
import pysam

hg38 = pysam.FastaFile('hg38.fa')

In [13]:
onehots = []

n = len(starts)
for i in range(n):
    seq = hg38.fetch(reference='chr{}'.format(chroms[i]), start=starts[i], end=ends[i]+1)
    onehot = dna_1hot(seq)
    onehots.append(onehot)

In [14]:
onehots = np.array(onehots)
onehots.shape #should be [n, 131072, 4]

(1308, 131072, 4)

In [16]:
#reshape and write output .npz of one-hot seqs
onehots = onehots.transpose(0,2,1) # reshape to (1308, 4, 131072)
np.savez('/content/drive/MyDrive/cs294_rep_attributions/onehot.npz', onehot=onehots)

In [35]:
test = np.load('/content/drive/MyDrive/cs294_rep_attributions/onehot.npz', mmap_mode='r')
print(test['onehot'].shape)

(1308, 4, 131072)


# Run tfmodisco-lite

In [42]:
# load files
for rep in range(1,6):
    attr = np.load('/content/drive/MyDrive/cs294_rep_attributions/rep{}.npz'.format(rep), mmap_mode='r')['grads']
    attr = attr.transpose(0,2,1)
    np.savez('attr_rep{}.npz'.format(rep), arr_0=attr)

ohe = np.load('/content/drive/MyDrive/cs294_rep_attributions/onehot.npz', mmap_mode='r')['onehot']
np.savez('ohe.npz', arr_0=ohe)

In [43]:
ohe = np.load('ohe.npz', mmap_mode='r')
for key in ohe.files:
    print(f"The shape of the one-hot encoding is {ohe[key].shape}")

attr = np.load('attr_rep1.npz'.format(rep), mmap_mode='r')
for key in attr.files:
    print(f"The shape of the attribution scores is {attr[key].shape}")

The shape of the one-hot encoding is (1308, 4, 131072)
The shape of the attribution scores is (1308, 4, 131072)


In [44]:
%%sh
mkdir -p modisco
modisco motifs -s ohe.npz -a attr_rep1.npz -n 2000 -o modisco/modisco_results_rep1.h5
modisco motifs -s ohe.npz -a attr_rep2.npz -n 2000 -o modisco/modisco_results_rep2.h5
modisco motifs -s ohe.npz -a attr_rep3.npz -n 2000 -o modisco/modisco_results_rep3.h5
modisco motifs -s ohe.npz -a attr_rep4.npz -n 2000 -o modisco/modisco_results_rep4.h5
modisco motifs -s ohe.npz -a attr_rep5.npz -n 2000 -o modisco/modisco_results_rep5.h5

In [45]:
%%sh
cd modisco
modisco report -i modisco_results_rep1.h5 -o report1/
modisco report -i modisco_results_rep2.h5 -o report2/
modisco report -i modisco_results_rep3.h5 -o report3/
modisco report -i modisco_results_rep4.h5 -o report4/
modisco report -i modisco_results_rep5.h5 -o report5/

In [56]:
from IPython.display import HTML
!cd modisco/report1/
HTML('modisco/report1/motifs.html')
#image paths seem to be broken, will download to render locally instead

pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev
pos_patterns.pattern_0,66,,
pos_patterns.pattern_1,65,,
pos_patterns.pattern_2,43,,
pos_patterns.pattern_3,37,,
pos_patterns.pattern_4,36,,
pos_patterns.pattern_5,33,,
pos_patterns.pattern_6,29,,
pos_patterns.pattern_7,28,,
pos_patterns.pattern_8,27,,
neg_patterns.pattern_0,48,,


In [63]:
%%sh
cd modisco
mkdir -p data
wget --quiet https://jaspar.genereg.net/download/data/2022/CORE/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt -O data/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt

In [64]:
%%bash
conda install --quiet -c bioconda meme

Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - meme


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    expat-2.6.2                |       h59595ed_0         134 KB  conda-forge
    icu-72.1                   |       hcb278e6_0        11.4 MB  conda-forge
    libarchive-3.7.2           |       h039dbb9_0         846 KB  conda-forge
    libexpat-2.6.2             |       h59595ed_0          72 KB  conda-forge
    libgfortran-ng-13.2.0      |       h69a702a_7          24 KB  conda-forge
    libgfortran5-13.2.0        |       hca663fb_7         1.4 MB  conda-forge
    libuv-1.44.2               |       hd590300_1         805 KB  conda-forge
    libxcrypt-4.4.36           |       hd590300_1          98 KB  conda-forge
    libxml2-2.11.5             |       h0

In [80]:
%%sh
cd modisco
modisco report -i modisco_results_rep1.h5 -o report1_jaspar/ -s report1_jaspar/ \
-m data/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt

In [86]:
%%sh
cd modisco
modisco report -i modisco_results_rep2.h5 -o report2_jaspar/ -s report2_jaspar/ \
-m data/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt

In [87]:
%%sh
cd modisco
modisco report -i modisco_results_rep3.h5 -o report3_jaspar/ -s report3_jaspar/ \
-m data/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt

In [88]:
%%sh
cd modisco
modisco report -i modisco_results_rep4.h5 -o report4_jaspar/ -s report4_jaspar/ \
-m data/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt

In [89]:
%%sh
cd modisco
modisco report -i modisco_results_rep5.h5 -o report5_jaspar/ -s report5_jaspar/ \
-m data/JASPAR2022_CORE_vertebrates_non-redundant_pfms_meme.txt

In [81]:
from IPython.display import HTML
HTML('modisco/report1_jaspar/motifs.html')
#image paths seem to be broken, will download to render locally instead

pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
pos_patterns.pattern_0,66,,,MA0750.2,0.017882,,MA0641.1,0.293496,,MA0686.1,0.424781,
pos_patterns.pattern_1,65,,,MA0641.1,0.015822,,MA0028.2,0.029499,,MA0765.3,0.029499,
pos_patterns.pattern_2,43,,,MA1513.1,0.283895,,MA1712.1,0.300993,,MA0599.1,0.300993,
pos_patterns.pattern_3,37,,,MA1713.1,0.001209,,MA1513.1,0.001209,,MA1961.1,0.001209,
pos_patterns.pattern_4,36,,,MA1961.1,0.000146,,MA1513.1,0.000146,,MA0079.5,0.000146,
pos_patterns.pattern_5,33,,,MA0606.2,1.000000,,MA1123.2,1.000000,,MA1972.1,1.000000,
pos_patterns.pattern_6,29,,,MA1981.1,0.007069,,MA1513.1,0.014379,,MA0516.3,0.014379,
pos_patterns.pattern_7,28,,,MA1961.1,0.001419,,MA1513.1,0.002249,,MA1653.1,0.002249,
pos_patterns.pattern_8,27,,,MA1967.1,0.418602,,MA0762.1,0.418602,,MA1941.1,0.439962,
neg_patterns.pattern_0,48,,,MA0742.2,0.011329,,MA0516.3,0.011329,,MA1513.1,0.011329,


In [90]:
!zip -r modisco.zip /content/modisco

updating: content/modisco/ (stored 0%)
updating: content/modisco/modisco_results_rep3.h5 (deflated 69%)
updating: content/modisco/report4/ (stored 0%)
updating: content/modisco/report4/trimmed_logos/ (stored 0%)
updating: content/modisco/report4/trimmed_logos/neg_patterns.pattern_0.cwm.fwd.png (deflated 2%)
updating: content/modisco/report4/trimmed_logos/pos_patterns.pattern_6.cwm.rev.png (deflated 5%)
updating: content/modisco/report4/trimmed_logos/neg_patterns.pattern_8.cwm.fwd.png (deflated 5%)
updating: content/modisco/report4/trimmed_logos/neg_patterns.pattern_5.cwm.fwd.png (deflated 2%)
updating: content/modisco/report4/trimmed_logos/neg_patterns.pattern_5.cwm.rev.png (deflated 2%)
updating: content/modisco/report4/trimmed_logos/neg_patterns.pattern_10.cwm.fwd.png (deflated 4%)
updating: content/modisco/report4/trimmed_logos/neg_patterns.pattern_4.cwm.fwd.png (deflated 2%)
updating: content/modisco/report4/trimmed_logos/pos_patterns.pattern_0.cwm.fwd.png (deflated 5%)
updating: c

In [91]:
from google.colab import files
files.download('modisco.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>